<a href="https://colab.research.google.com/github/MikeSalnikov/Data-analysis-algorithms/blob/main/HW7_web7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 7

К алгоритму kNN, представленному на уроке, реализуйте добавление весов для соседей по любому из показанных на уроке принципов. Сравните точность нового алгоритма с исходным при разных значениях k.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

Загружаю один из "игрушечных" датасетов из sklearn (классификация цветов Ириса):

In [2]:
X, y = load_iris(return_X_y=True)

# для наглядности беоу только первые два признака (всего в датасете их 4)
X = X[:, :2]

Разделяю выборку на обучающую и тестовую: 

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((120, 2), (30, 2))

Использую евклидову метрику. Реализую функцию для ее подсчета: 

In [4]:
def e_metrics(x1, x2):
    
    distance = np.sum(np.square(x1 - x2))

    return np.sqrt(distance)

Реализую алгоритм поиска k ближайших соседей: 

In [5]:
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # записываю в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаю словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # сортирую список и среди первых k элементов подсчитываю частоту появления разных классов
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += 1

        # записываю в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

Напишу функцию для вычисления точности: 

In [6]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

Проверяю работу алгоритма при различных k: 

In [7]:
k = [1, 2, 3, 5, 10, 50]

result1 = {}

for el in k:
    y_pred = knn(X_train, y_train, X_test, el)
    result1[el] = round(accuracy(y_pred, y_test), 3)

Выведу значения точности при разных k: 

In [8]:
result1

{1: 0.7, 2: 0.733, 3: 0.733, 5: 0.733, 10: 0.767, 50: 0.7}

Реализую алгоритм поиска k ближайших соседей с добавлением весов по формуле: 

### $w(d) = \frac{1}{d+a}$

In [9]:
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # записываю в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаю словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # сортирую список и среди первых k элементов посчитываю сумму весов классов ближайших соседей
        a = 1        
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += 1 / (d[0] + a) # вместо инкремента прибавляю вес признака в словарь по ближайшим соседям по вышеуказанной формуле

        # записываю в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [10]:
k = [1, 2, 3, 5, 10, 50]

result2 = {}

for el in k:
    y_pred = knn(X_train, y_train, X_test, el)
    result2[el] = round(accuracy(y_pred, y_test), 3)

Выведу новые значения точности при разных k: 

In [11]:
result2

{1: 0.7, 2: 0.7, 3: 0.733, 5: 0.733, 10: 0.733, 50: 0.7}

Реализую алгоритм поиска k ближайших соседей с добавлением весов по формуле: 

### $w(d) = q^{d}$,   $q \in (0,1)$

In [12]:
def knn(x_train, y_train, x_test, k, q):  # добавлен аргумент q, устанавливаемый при запуске функции вручную
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes= {class_item: 0 for class_item in set(y_train)}

        # сортирую список и среди первых k элементов посчитываю сумму весов классов ближайших соседей
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += q ** d[0]  # вместо инкремента прибавляю вес признака в словарь по ближайшим соседям по вышеуказанной формуле

        # записываю в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [13]:
k = [1, 2, 3, 5, 10, 50]

result3 = {}

for el in k:
    y_pred = knn(X_train, y_train, X_test, el, q=0.1)
    result3[el] = round(accuracy(y_pred, y_test), 3)

Выведу новые значения точности при разных k: 

In [14]:
result3

{1: 0.7, 2: 0.7, 3: 0.733, 5: 0.733, 10: 0.733, 50: 0.733}

Сравниваю точность новых алгоритмов с исходным при разных значениях k: 

In [15]:
result1, result2, result3

({1: 0.7, 2: 0.733, 3: 0.733, 5: 0.733, 10: 0.767, 50: 0.7},
 {1: 0.7, 2: 0.7, 3: 0.733, 5: 0.733, 10: 0.733, 50: 0.7},
 {1: 0.7, 2: 0.7, 3: 0.733, 5: 0.733, 10: 0.733, 50: 0.733})

При увеличении k наблюдается сначала рост точности, затем её стабилизация, затем падение (за исключением третьего алгоритма). 

Можно заметить, что именно первый алгоритм показывает наилучшие результаты. 